# Disclaimer: Gridsearches are run using 50 CPU threads, so don't run unless you want to wait eons

## Model Tuning

Here we will evalute our baseline model and attempt to build a better model using sentiment analysis and a voting classifier using our previous models as part of the voting classifier if we have time. 

In [1]:
# import our libraries

import pandas as pd
import numpy as np
import re
import textwrap

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.experimental import enable_hist_gradient_boosting, enable_halving_search_cv
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, HalvingRandomSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, \
VotingClassifier, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, FunctionTransformer

from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import dataset

translated_df = pd.read_csv('../data/translated.csv')
translated_df.head()

,subreddit,merged_text
0,1,Fix to not being able to attack? Has anybody f...
1,1,Attack glitch during Regi raids 2 raids today ...
2,1,[Bug?] Can’t seem to earn or collect pokecoins...
3,1,"[Bug?] AR suddenly freezes Using an iPhone 11,..."
4,1,3 hour incense event personal results For any ...


In [3]:
# remove duplicate entries where text is the same
translated_df = translated_df.drop_duplicates('merged_text')

Now we get get sentiment scores for our text.

In [4]:
# thanks to Hov
# instantiate the sentiment analyzer
sia = SentimentIntensityAnalyzer()
def get_compound_sentiment(post):
    return sia.polarity_scores(post)['compound']

translated_df['sentiment'] = translated_df['merged_text'].apply(get_compound_sentiment)

In [5]:
get_text_data = FunctionTransformer(lambda x: x['merged_text'], validate = False)
get_numeric_data = FunctionTransformer(lambda x: x[['sentiment']], validate = False)

In [6]:
# create baseline model
translated_df['subreddit'].value_counts()

1    10611
0     8642
Name: subreddit, dtype: int64

We will guess that all posts are from TheSilphRoad subreddit as it is our majority class.

In [7]:
# create train_test_split to evaluate our baseline model
X = translated_df[['merged_text', 'sentiment']]
y = translated_df['subreddit']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# create predictions for train and test set
y_train_preds = np.full_like(y_train, 1)
y_test_preds = np.full_like(y_test, 1)

print(f'Baseline accuracy on training data: {round(accuracy_score(y_train, y_train_preds), 2)}')
print(f'Baseline accuracy on testing data: {round(accuracy_score(y_test, y_test_preds), 2)}')

Baseline accuracy on training data: 0.55
Baseline accuracy on testing data: 0.55


Here we see that we have an baseline accuracy of 55% on both our training and testing data which is expected as about 55% of the posts we have are from TheSilphRoad subreddit and we used stratify for our train/test split. 

In [9]:
# create a custom stop words list to remove all similar words we found in the previous notebook

# start with the base english stopwords
new_stopwords = stopwords.words('english')

# add stopwords that will easily identify a silph post
# also added stopwords that are common across both subreddits and stop words as a result and lemmatizing and stemming
custom_words = ['silph', 'road', 'silphroad', 'thesilphroad', 'pokemon', 'go', 'get', 'one', 'like', 'would', 'know', 'time', 'game', 'shiny', 
               'https', 'raid', 'anyone', 'got', 'new', 'event', 'day', 'level', 'even', 'com', 'raids', 'still', 'people', 'also', 'since',
               'use', 'catch', 'amp', 'see', 'want', 'could', 'first', 'research', 'shadow', 'think', 'else', 'way', 'niantic', 'make', 
               'back', 'really', 'need', 'eggs', 'community', 'something', 'much', 'good', 'able', "'d", "'ll", "'re", "'s", "'ve", 'abl', 
                'abov', 'ani', 'anyon', 'becaus', 'befor', 'commun', 'doe', 'dure', 'egg', 'els', 'ha', 'hi', 'http', 'might', 'must', 
                "n't", 'onc', 'onli', 'ourselv', 'peopl', 'realli', 'sha', 'shini', 'sinc', 'someth', 'themselv', 'thi', 'veri', 'wa', 
                'whi', 'wo', 'yourselv', 'becau', 'el']

new_stopwords.extend(custom_words)

In [10]:
# custom preprocessing of text 
# thanks to the following article for the code: https://kavita-ganesan.com/how-to-use-countvectorizer/#CountVectorizer-Plain-and-Simple

lemmatizer = WordNetLemmatizer()
def lemma_preprocessor(text):
    text = text.lower()
    text = re.sub("\\W", " ", text) # remove special characters
    text = re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ", text) # normalize certain words
    
    #stem words
    words = re.split("\\s+", text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

porter_stemmer = PorterStemmer()
def stem_preprocessor(text):
    text = text.lower()
    text = re.sub("\\W", " ", text) # remove special characters
    text = re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ", text) # normalize certain words
    
    #stem words
    words = re.split("\\s+", text)
    stemmed_words = [porter_stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [11]:
# generate our pipelines
cvec_sentiment_pipe = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('ss', StandardScaler())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('cvec', CountVectorizer(stop_words = new_stopwords, strip_accents = 'unicode'))
            ]))
    ])),
    ('densify', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
    ('vote', VotingClassifier([
        ('logr', LogisticRegression(solver = 'liblinear', max_iter = 10000)),
        ('rforest', RandomForestClassifier()),
        ('bag', BaggingClassifier()),
        ('knn', KNeighborsClassifier()),
        ('etree', ExtraTreesClassifier()),
        ('xgb', XGBClassifier(use_label_encoder = False, n_jobs = 1, eval_metric = "logloss")),
        ('svc', SVC()), 
        ('histboost', HistGradientBoostingClassifier())
    ]))
])

tfidf_sentiment_pipe = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('ss', StandardScaler())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('tfidf', TfidfVectorizer(stop_words = new_stopwords, strip_accents = 'unicode')),
            ]))
    ])),
    ('densify', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
    ('vote', VotingClassifier([
        ('logr', LogisticRegression(solver = 'liblinear', max_iter = 10000)),
        ('rforest', RandomForestClassifier()),
        ('bag', BaggingClassifier()),
        ('knn', KNeighborsClassifier()),
        ('etree', ExtraTreesClassifier()),
        ('xgb', XGBClassifier(use_label_encoder = False, n_jobs = 1, eval_metric = "logloss")),
        ('svc', SVC()),
        ('histboost', HistGradientBoostingClassifier())
    ]))
])

In [12]:
# create gridsearch parameters

cvec_sentiment_params = {
    'features__text_features__cvec__preprocessor': [lemma_preprocessor, stem_preprocessor],
    'features__text_features__cvec__ngram_range': [(1,1), (1,2), (1,3), (2,3),(3,3)],
    'features__text_features__cvec__max_df': np.linspace(0.8, 1.0, 10),
    'features__text_features__cvec__min_df': range(1,4),
    'features__text_features__cvec__max_features': range(10000, 100000, 100),
    'vote__logr__penalty': ['l1', 'l2'],
    'vote__logr__C': np.linspace(0.0001, 1, 50),
    'vote__rforest__n_estimators': range(50, 1000, 25),
    'vote__rforest__max_depth': range(10, 101, 4),
    'vote__rforest__max_features': ['sqrt', 'log2'],
    'vote__xgb__n_estimators': range(10, 1000, 100), 
    'vote__xgb__learning_rate': [0.01, 0.1, 0.3, 0.7],
    'vote__xgb__max_depth': [2, 5, 10], 
    'vote__xgb__min_child_weight': range(2, 100, 4), 
    'vote__histboost__learning_rate': np.linspace(0.01, 1, 50),
    'vote__histboost__max_iter': range(100, 10000, 200),
    'vote__histboost__max_depth': range(10, 101, 4),
}

tfidf_sentiment_params = {
    'features__text_features__tfidf__preprocessor': [lemma_preprocessor, stem_preprocessor],
    'features__text_features__tfidf__ngram_range': [(1,1), (1,2), (1,3), (2,3),(2,2),(3,3)],
    'features__text_features__tfidf__max_df': np.linspace(0.8, 1.0, 10),
    'features__text_features__tfidf__min_df': range(1,4),
    'features__text_features__tfidf__max_features': range(2000, 100000, 100),
    'vote__logr__penalty': ['l1', 'l2'],
    'vote__logr__C': np.linspace(0.0001, 1, 1000),
    'vote__rforest__n_estimators': range(50, 1000, 50),
    'vote__rforest__max_depth': range(10, 101, 2),
    'vote__rforest__max_features': ['sqrt', 'log2'],
    'vote__xgb__n_estimators': range(10, 1000, 50), 
    'vote__xgb__learning_rate': [0.01, 0.1, 0.3, 0.7],
    'vote__xgb__max_depth': [2, 5, 10], 
    'vote__xgb__min_child_weight': range(2, 100, 2), 
    'vote__histboost__learning_rate': np.linspace(0.01, 1, 100),
    'vote__histboost__max_iter': range(100, 10000, 100),
    'vote__histboost__max_depth': range(10, 101, 2),
}

In [13]:
cvec_sentiment_gs = HalvingRandomSearchCV(estimator = cvec_sentiment_pipe,
                                param_distributions = cvec_sentiment_params,
                                n_candidates = 10000,
                                verbose = 10, 
                                cv = 5, 
                                n_jobs = 50)

tfidf_sentiment_gs = HalvingRandomSearchCV(estimator = tfidf_sentiment_pipe,
                                param_distributions = tfidf_sentiment_params,
                                n_candidates = 10000,
                                verbose = 10, 
                                cv = 5, 
                                n_jobs = 50)

In [ ]:
# put all gridsearches into a list
gridsearches = [cvec_sentiment_gs, tfidf_sentiment_gs]

# create dictionary of gridsearches
gridsearch_dict = {0: 'Count Vectorizer Sentiment Voting', 1: 'TFIDF Sentiment Voting'}

# thanks to the following article for the code below: https://www.kdnuggets.com/2018/01/managing-machine-learning-workflows-scikit-learn-pipelines-part-3.html

for i, gs in enumerate(gridsearches):
    print('\nEstimator: %s' % gridsearch_dict[i])
    gs.fit(X_train, y_train)
    print('Best Parameters: %s' % gs.best_params_)
    print('Training accuracy: %.3f' % gs.best_score_)
    y_pred = gs.predict(X_test)
    print('Test accurracy: %.3f' % accuracy_score(y_test, y_pred))


Estimator: Count Vectorizer Sentiment Voting
n_iterations: 7
n_required_iterations: 9
n_possible_iterations: 7
min_resources_: 20
max_resources_: 15402
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 10000
n_resources: 20
Fitting 5 folds for each of 10000 candidates, totalling 50000 fits


/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.75       0.55       0.61666667 ...        nan 0.48333333 0.48333333]
  warnings.warn(
/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [0.95   0.9125 0.8875 ...    nan 0.8875 0.925 ]
  warnings.warn(


----------
iter: 1
n_candidates: 3334
n_resources: 60
Fitting 5 folds for each of 3334 candidates, totalling 16670 fits


/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.75       0.55       0.61666667 ... 0.43787879 0.49090909        nan]
  warnings.warn(
/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [0.95       0.9125     0.8875     ... 0.80230496 0.80691489        nan]
  warnings.warn(


----------
iter: 2
n_candidates: 1112
n_resources: 180
Fitting 5 folds for each of 1112 candidates, totalling 5560 fits
----------
iter: 3
n_candidates: 371
n_resources: 540
Fitting 5 folds for each of 371 candidates, totalling 1855 fits


/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.75       0.55       0.61666667 ... 0.54714286 0.5468254  0.5531746 ]
  warnings.warn(
/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [0.95       0.9125     0.8875     ... 0.84950466 0.88021562 0.85930458]
  warnings.warn(


----------
iter: 4
n_candidates: 124
n_resources: 1620
Fitting 5 folds for each of 124 candidates, totalling 620 fits


/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.75       0.55       0.61666667 ... 0.58459675 0.56036691 0.56780893]
  warnings.warn(
/home/jesse/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [0.95       0.9125     0.8875     ... 0.95411833 0.97683144 0.95644603]
  warnings.warn(


We are still waiting on this extensive gridsearch to finish and may not be finished by the deadline to turn this in. 